In [23]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
import warnings
import numpy as np
import cv2 as cv
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping, ModelCheckpoint
warnings.simplefilter(action = 'ignore', category = FutureWarning)

background = None
accumulated_weight = 0.5

ROI_top = 15
ROI_bottom = 350
ROI_right = 150
ROI_left = 550

def cal_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    cv.accumulateWeighted(frame, background, accumulated_weight) # to calculate average of frames, frame type must be of float values else it won`t work

def segment_hand(frame, threshold = 25):
    global background
    
    diff = cv.absdiff(background.astype('uint8'), frame)
    _, thresholded = cv.threshold(diff, threshold, 255, cv.THRESH_BINARY)
    
    contours, hierarchy = cv.findContours(thresholded.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        hand_seg_max_count = max(contours, key = cv.contourArea)
        return (thresholded, hand_seg_max_count)

cam = cv.VideoCapture(0)

num_frames = 0
element = 10
num_imgs_taken = 0

while True:
    
    ret, frame = cam.read()
    
    cv.waitKey(1)
    frame = cv.flip(frame, 1) # 0 for x axis, 1 for y axis, -1 for both axis..
    cv.imshow("wind", frame)

    frame_copy = frame.copy()
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
    
    gray_image = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    gray_image = cv.GaussianBlur(gray_image, (9,9), 0)
    
    if num_frames < 60:
        cal_accum_avg(gray_image, accumulated_weight)
        if num_frames <= 59:
            cv.putText(frame_copy, " FETCHING BACKGROUND...PLEASE WAIT ", (80, 400), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    elif num_frames <= 300:
        hand = segment_hand(gray_image)
        
        cv.putText(frame_copy, " Make hand gesture for.. " + str(element), (200, 400), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        
        if hand is not None: #is compares the id values of variables..returns true if two variables refer to the same object..
            thresholded, hand_segment = hand
            
            cv.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255,0,0), 1)
            cv.putText(frame_copy, str(num_frames) + " for " + str(element), (70, 45), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 1)
            cv.imshow("Thresholded_image", thresholded)
    else:
        hand = segment_hand(gray_image)
        
        if hand is not None:
            thresholded, hand_segment = hand
            
            cv.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255,0,0), 1)
            cv.putText(frame_copy, str(num_frames), (70, 45), cv.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1)
            cv.putText(frame_copy, str(num_imgs_taken) + "images for" + str(element), (200, 400), cv.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1)
            cv.imshow("Thresholded image", thresholded)
            
            if num_imgs_taken <= 40:
                cv.imwrite(r"C:\Users\shubh\sign language detection\test\\" + str(element) + "\\" + str(num_imgs_taken) + ".bmp", thresholded)
            else:
                break
            num_imgs_taken += 1
        
        else:
            cv.putText(frame_copy, 'No hand detected..try again', (200, 400), cv.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1)
    
    cv.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255, 100, 10), 5)
    cv.putText(frame_copy, "Put your hand on my shou-..ahm! here, I mean", (10, 20), cv.FONT_HERSHEY_SIMPLEX, 1, (100, 200, 30), 4)
    num_frames += 1
    cv.imshow("Sign detection", frame_copy)
    
    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()
cam.release()